In [7]:
# Import necesary libraries
import yfinance as yf
import datetime as dt
import datetime as dt
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
from stocktrends import Renko
from signals import momentum_signals
file = r"C:\Users\8prab\Google Drive\Work\xooca-git\trading_related\BHARTIARTL.NS\hist_2010-01-01_2020-06-19.csv"

ModuleNotFoundError: No module named 'signals'

In [2]:
df = pd.read_csv(file,encoding='latin-1',index_col=['Unnamed: 0'])

In [3]:
df

,Open,High,Low,Close*,Adj. close**,Volume
09-Apr-2010,286.47,290.60,281.88,282.71,266.75,"63,64,752"
08-Apr-2010,292.90,295.19,284.64,287.30,271.08,"67,50,418"
07-Apr-2010,291.06,299.14,288.03,294.18,277.58,"56,64,078"
06-Apr-2010,291.52,292.71,286.29,290.56,274.16,"40,53,513"
05-Apr-2010,281.88,292.12,277.75,290.19,273.81,"1,05,43,536"
...,...,...,...,...,...,...
01-Apr-2020,440.00,440.00,412.40,421.35,421.35,"94,69,123"
31-Mar-2020,441.00,447.00,424.00,440.90,440.90,"1,67,45,069"
30-Mar-2020,443.85,449.30,428.00,431.05,431.05,"1,00,22,553"
27-Mar-2020,470.70,474.80,437.50,448.95,448.95,"1,60,76,572"


In [4]:
from talib.abstract import *
inputs = {
    'open': np.array(df['Open']),
    'high': np.array(df['High']),
    'low': np.array(df['Low']),
    'close': np.array(df['Close*']),
    'volume': np.array(df['Volume'])
}
output = SMA(inputs, timeperiod=25) # calculate on close prices by default
#output = SMA(inputs, timeperiod=25, price='open') # calculate on opens

Exception: input array type is not double

In [5]:
np.array(df['Open'])

array(['286.47', '292.90', '291.06', ..., '443.85', '470.70', '426.50'],
      dtype=object)

In [6]:
np.random.random(100)

array([0.21079355, 0.31520061, 0.1415485 , 0.5776051 , 0.09713636,
       0.92247833, 0.90372242, 0.60003721, 0.44639716, 0.53192807,
       0.07825677, 0.53375275, 0.05145896, 0.49026858, 0.76477539,
       0.19435274, 0.3716176 , 0.47007946, 0.81651848, 0.16324662,
       0.46827888, 0.23067051, 0.26563278, 0.89288331, 0.71314477,
       0.38805942, 0.7331028 , 0.88153698, 0.32394455, 0.82828624,
       0.52460925, 0.37090481, 0.17278667, 0.88376262, 0.97974744,
       0.44167047, 0.21914929, 0.78502398, 0.26088931, 0.93290961,
       0.70538553, 0.5191886 , 0.62775706, 0.21130212, 0.06248358,
       0.24682621, 0.25966045, 0.51407079, 0.22968238, 0.16842323,
       0.03773623, 0.398247  , 0.10128631, 0.79434818, 0.87731647,
       0.54742605, 0.59869385, 0.87580599, 0.37603137, 0.54661856,
       0.78399056, 0.24338935, 0.24171494, 0.80879699, 0.74015283,
       0.87915383, 0.99138473, 0.89784905, 0.08301372, 0.70274316,
       0.58839205, 0.17897888, 0.89485087, 0.99919249, 0.77455

In [19]:
def MACD(DF,a,b,c):
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    df = DF.copy()
    df = df[df['Open']!='-']
    df['High'] = df['High'].astype('float')
    df['Low'] = df['Low'].astype('float')
    df['Adj. close**'] = df['Adj. close**'].astype('float')
    df["MA_Fast"]=df["Adj. close**"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["Adj. close**"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    #Signal-MACD : Negative is Bearish and Positive is Bullish
    df["Signal-MACD"] = df["Signal"] - df["MACD"]
    df.dropna(inplace=True)
    return df

# Visualization - plotting MACD/signal along with close price and volume for last 100 data points
df = MACD(df, 12, 26, 9)

In [20]:
def rsi(df, n):
    "function to calculate RSI"
    df = df[df['Open']!='-']
    df['High'] = df['High'].astype('float')
    df['Low'] = df['Low'].astype('float')
    df['Adj. close**'] = df['Adj. close**'].astype('float')
    delta = df["Adj. close**"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    # RSI above 70 is overbought and below 30 is oversold
    df['RSI'] = 100 - 100 / (1+rs)
    return df
rsi(df,10)

,Unnamed: 0,Open,High,Low,Close*,Adj. close**,Volume,MA_Fast,MA_Slow,MACD,Signal,Signal-MACD,RSI
33,18-Feb-2010,256.17,261.13,253.42,258.83,244.22,"76,41,742",245.998725,251.164068,-5.165344,-4.981367,0.183977,NaN
34,17-Feb-2010,251.03,258.70,251.03,256.31,241.84,"1,75,69,953",245.357067,250.423295,-5.066228,-5.000380,0.065847,NaN
35,16-Feb-2010,262.60,265.81,247.31,249.38,235.30,"4,90,59,303",243.806035,249.228209,-5.422174,-5.092667,0.329508,NaN
36,15-Feb-2010,291.06,291.06,261.13,262.23,247.43,"2,06,63,298",244.364723,249.086809,-4.722086,-5.013081,-0.290996,NaN
37,11-Feb-2010,289.68,291.89,285.32,288.77,272.47,"36,73,279",248.696193,250.917170,-2.220978,-4.422175,-2.201198,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088,01-Apr-2020,440.00,440.00,412.40,421.35,421.35,"94,69,123",472.981740,490.610426,-17.628686,-8.650115,8.978571,25.993528
2089,31-Mar-2020,441.00,447.00,424.00,440.90,440.90,"1,67,45,069",468.046088,486.928172,-18.882084,-10.696509,8.185576,35.277679
2090,30-Mar-2020,443.85,449.30,428.00,431.05,431.05,"1,00,22,553",462.354382,482.789049,-20.434666,-12.644140,7.790526,32.984369
2091,27-Mar-2020,470.70,474.80,437.50,448.95,448.95,"1,60,76,572",460.292170,480.282452,-19.990283,-14.113369,5.876914,40.691445


In [21]:
def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df = df[df['Open']!='-']
    df['High'] = df['High'].astype('float')
    df['Low'] = df['Low'].astype('float')
    df['Adj. close**'] = df['Adj. close**'].astype('float')
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj. close**'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj. close**'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2


def BollBnd(DF,n):
    "function to calculate Bollinger Band"
    df = DF.copy()
    df = df[df['Open']!='-']
    df = df[df['Open']!='-']
    df['Adj. close**'] = df['Adj. close**'].astype('float')
    df["MA"] = df['Adj. close**'].rolling(n).mean()
    df["BB_up"] = df["MA"] + 2*df['Adj. close**'].rolling(n).std(ddof=0) 
    df["BB_dn"] = df["MA"] - 2*df['Adj. close**'].rolling(n).std(ddof=0)
    df["BB_width"] = df["BB_up"] - df["BB_dn"]
    df.dropna(inplace=True)
    return df

In [22]:
def slope(DF,n=5):
    "function to calculate the slope of regression line for n consecutive points on a plot"
    df = DF.copy()
    df = df[df['Open']!='-']
    df['High'] = df['High'].astype('float')
    df['Low'] = df['Low'].astype('float')
    df['Adj. close**'] = df['Adj. close**'].astype('float')
    ser = df["Adj. close**"]
    ser = (ser - ser.min())/(ser.max() - ser.min())
    x = np.array(range(len(ser)))
    x = (x - x.min())/(x.max() - x.min())
    slopes = [i*0 for i in range(n-1)]
    for i in range(n,len(ser)+1):
        y_scaled = ser[i-n:i]
        x_scaled = x[i-n:i]
        x_scaled = sm.add_constant(x_scaled)
        model = sm.OLS(y_scaled,x_scaled)
        results = model.fit()
        slopes.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(slopes))))
    df['slope'] = np.array(slope_angle)
    return df

In [23]:
slope(df,n=5)

,Unnamed: 0,Open,High,Low,Close*,Adj. close**,Volume,MA_Fast,MA_Slow,MACD,Signal,Signal-MACD,slope
33,18-Feb-2010,256.17,261.13,253.42,258.83,244.22,"76,41,742",245.998725,251.164068,-5.165344,-4.981367,0.183977,0.000000
34,17-Feb-2010,251.03,258.70,251.03,256.31,241.84,"1,75,69,953",245.357067,250.423295,-5.066228,-5.000380,0.065847,0.000000
35,16-Feb-2010,262.60,265.81,247.31,249.38,235.30,"4,90,59,303",243.806035,249.228209,-5.422174,-5.092667,0.329508,0.000000
36,15-Feb-2010,291.06,291.06,261.13,262.23,247.43,"2,06,63,298",244.364723,249.086809,-4.722086,-5.013081,-0.290996,0.000000
37,11-Feb-2010,289.68,291.89,285.32,288.77,272.47,"36,73,279",248.696193,250.917170,-2.220978,-4.422175,-2.201198,88.260739
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088,01-Apr-2020,440.00,440.00,412.40,421.35,421.35,"94,69,123",472.981740,490.610426,-17.628686,-8.650115,8.978571,-89.373436
2089,31-Mar-2020,441.00,447.00,424.00,440.90,440.90,"1,67,45,069",468.046088,486.928172,-18.882084,-10.696509,8.185576,-88.804557
2090,30-Mar-2020,443.85,449.30,428.00,431.05,431.05,"1,00,22,553",462.354382,482.789049,-20.434666,-12.644140,7.790526,-88.239773
2091,27-Mar-2020,470.70,474.80,437.50,448.95,448.95,"1,60,76,572",460.292170,480.282452,-19.990283,-14.113369,5.876914,88.191155


In [24]:
def rsi(df, n):
    "function to calculate RSI"
    delta = df["Adj Close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    return 100 - 100 / (1+rs)

In [25]:
#df = BollBnd(ohlcv,20).iloc[-100:,[-4,-3,-2]].plot(title="Bollinger Band")


In [30]:

def renko_DF(DF):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    print(df.columns)
    df.rename(columns = {"Unnamed: 0" : "date", "Open" : "open","High" : "high","Low" : "low","Close*":"act_close","Adj. close**" : "close", "Volume" : "volume"}, inplace = True)
    df = df[['date','high','low','open','close','volume']]
    df2 = Renko(df)
    print(df.head())
    df2.brick_size = round(ATR(DF,120)["ATR"][-1],0)
    renko_df = df2.get_ohlc_data() #if using older version of the library please use get_bricks() instead
    return renko_df

In [63]:
#print(df.columns)
df = pd.read_csv(file,encoding='latin-1')
df = df[df['Open']!='-']
df['High'] = df['High'].astype('float')
df['Low'] = df['Low'].astype('float')
df['Adj. close**'] = df['Adj. close**'].astype('float')
df1 = df.copy()
df.rename(columns = {"Unnamed: 0" : "date", "Open" : "open","High" : "high","Low" : "low","Close*":"act_close","Adj. close**" : "close", "Volume" : "volume"}, inplace = True)
df = df[['date','high','low','open','close','volume']]

In [64]:
df2 = Renko(df)

In [65]:
df.columns

Index(['date', 'high', 'low', 'open', 'close', 'volume'], dtype='object')

In [66]:
#ATR(df1,120)["ATR"]

In [73]:
df2.brick_size

2085

In [75]:
df2 = Renko(df)
#df2.brick_size = round(ATR(df1,120)["ATR"],0)
df2.brick_size = round(ATR(df1,120)["ATR"].shape[-1],0)
df2.get_ohlc_data()    

,date,open,high,low,close,uptrend
0,09-Apr-2010,-2085,0.0,-2085.0,0.0,True


In [31]:
renko_DF(df)

Index(['Unnamed: 0', 'Open', 'High', 'Low', 'Close*', 'Adj. close**', 'Volume',
       'MA_Fast', 'MA_Slow', 'MACD', 'Signal', 'Signal-MACD'],
      dtype='object')
           date    high     low    open   close       volume
33  18-Feb-2010  261.13  253.42  256.17  244.22    76,41,742
34  17-Feb-2010  258.70  251.03  251.03  241.84  1,75,69,953
35  16-Feb-2010  265.81  247.31  262.60  235.30  4,90,59,303
36  15-Feb-2010  291.06  261.13  291.06  247.43  2,06,63,298
37  11-Feb-2010  291.89  285.32  289.68  272.47    36,73,279


KeyError: -1

In [29]:
plt.subplot(311)
plt.plot(df.iloc[-100:,4])
plt.title('MSFT Stock Price')
plt.xticks([])

plt.subplot(312)
plt.bar(df.iloc[-100:,5].index, df.iloc[-100:,5].values)
plt.title('Volume')
plt.xticks([])

plt.subplot(313)
plt.plot(df.iloc[-100:,[-2,-1]])
plt.title('MACD')
plt.legend(('MACD','Signal'),loc='lower right')

plt.show()


# Visualization - Using object orient approach
# Get the figure and the axes
fig, (ax0, ax1) = plt.subplots(nrows=2,ncols=1, sharex=True, sharey=False, figsize=(10, 6), gridspec_kw = {'height_ratios':[2.5, 1]})
df.iloc[-100:,4].plot(ax=ax0)
ax0.set(ylabel='Adj Close')

df.iloc[-100:,[-2,-1]].plot(ax=ax1)
ax1.set(xlabel='Date', ylabel='MACD/Signal')

# Title the figure
fig.suptitle('Stock Price with MACD', fontsize=14, fontweight='bold')


IndexingError: Too many indexers